In [ ]:
# optional, only for Jupyter
%matplotlib notebook

# General libraries
import numpy as np                # to deal with arrays, vectors, matrices...
import matplotlib.pyplot as plt   # to plot the data
import matplotlib.gridspec as gridspec

# Tensorflow
import os
HOME = os.getenv('HOME')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to get rid of the TF compilation warnings
import tensorflow as tf
from tensorflow.keras import models, datasets
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [ ]:
# Only because my system-wide config is tuned, you don't need these lines
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 5,3
mpl.rcParams['font.size'] = 6.0

In [ ]:
# Download data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
# cast to float32 to save ram
train_images = tf.cast(train_images, tf.float32)
test_images  = tf.cast(test_images, tf.float32)
inp_shape = train_images.shape[1:]

print(train_images.shape)
class_names = ['airplane', 'car', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
print(inp_shape)
print(type(train_images))
print(train_images.dtype)

In [ ]:
# n Random examples
n = 3
samples = np.random.choice(np.array(range(train_images.shape[0])), n*n)

fig, ax = plt.subplots(figsize=(4,4))
gs = gridspec.GridSpec(n, n)
axs = []
for i in range(n):
    for j in range(n):
        axs.append( plt.subplot(gs[i,j]) )

for i in range(len(samples)):
    ax = axs[i]
    ind = samples [i]
    img = train_images[ind,:,:]
    label = class_names[train_labels[ind][0]]
    #predicted = class_names[np.argmax(predictions[ind])]
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel(label)
plt.show()

In [ ]:
# model = models.Sequential()
# model.add(Flatten(input_shape=inp_shape))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(10, activation='softmax'))

# model.summary()

In [ ]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

The last layer was Dense with no activation, so the last activation was a ReLu function which ranges from 0 to inf, so the error must account for that. Two options:
- add a *softmax* activation in the last layer
- use "*from_logits*" in the error

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', #SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=0)
print(f'Pre-training accuracy: {test_acc*100:.2f}%')
from time import time
told = time()
history = model.fit(train_images, train_labels, epochs=15,
                    validation_data=(test_images, test_labels),
                    #validation_split=0.2)
                    verbose=1)
print(f'Trained in: {time()-told}s')

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f'Training accuracy: {test_acc*100:.2f}%')

In [ ]:
# plot learning curve
err = history.history['loss']
val_err = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plots grid
fig, ax = plt.subplots(figsize=(5,5))
gs = gridspec.GridSpec(2, 1)
ax0 = plt.subplot(gs[0, 0])
ax1 = plt.subplot(gs[1, 0])

# Loss plots
ax0.plot(err, label='Train')
ax0.plot(val_err, label='Test')
ax0.set_ylabel('Loss')

# Accuracy plots
ax1.plot(acc,label='Train')
ax1.plot(val_acc,label='Test')
ax1.set_ylabel('Accuracy')

# General settings
ax0.set_title('Learning curve')
plt.show()

In [ ]:
# Show Results
predictions = model.predict(test_images)

In [ ]:
# n Random examples
n = 3
samples = np.random.choice(np.array(range(test_images.shape[0])), n*n)

fig, ax = plt.subplots(figsize=(4,4))
gs = gridspec.GridSpec(n, n)
axs = []
for i in range(n):
    for j in range(n):
        axs.append( plt.subplot(gs[i,j]) )

for i in range(len(samples)):
    ax = axs[i]
    ind = samples [i]
    img = test_images[ind,:,:]
    label = test_labels[ind]
    predicted = class_names[np.argmax(predictions[ind])]
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_ylabel(predicted)
plt.show()

In [ ]:
# model.save('cifar.h5')